In [1]:
import pandas as pd
import tweepy
import requests
import os
import json
import time

## Gather data

Read in provided data to a dataframe.

In [2]:
ea = pd.read_csv('twitter-archive-enhanced.csv') # Read in enhanced twitter archive

Now use the requests library to programmatically download the `image-predictions.tsv` file from 

In [3]:
r = requests.get(
    'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

Write image predictions data in to a `.tsv` file.

In [4]:
with open('image-predictions.tsv', 'w', encoding='utf-8') as file: 
    file.write(str(r.text))

Read `.tsv` file in to dataframe `ip`.

In [5]:
ip = pd.read_csv('image-predictions.tsv', sep = '\t')

In [6]:
ip.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [7]:
ea.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


Grab all the tweet ids.

In [8]:
ids = ea['tweet_id']

Cycle through all the tweet ids, and grab their JSON data from the Twitter API using `tweepy` and `json`.

Many tweets have been deleted, and so cause an error to be thrown.

In [9]:
import tweepy

consumer_key = '2VDcRqXbcihwqtob3KyCHR4HA'
consumer_secret = 'Q2eRUisXRiu5mpFgakTyUQwpiIAGy4xHRC9fNW8I0FFCckGCdE'
access_token = '773745310661275652-rAdthEP5lVG6yiF4o2OydJnd3WGWryV'
access_secret = 'QbAWTTc2yah7G8GOtFQxAgGKH9iJT0liT1qy3zPvqhT8N'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True,
                wait_on_rate_limit_notify = True)

Grab all the JSON data from tweepy and store it in `tweet_json.txt`. If an error is thrown when we try to access a tweet this probably means the tweet was deleted. The `try` `except` block allows us to continue on despite errors.

In [ ]:
with open('tweet_json.txt', mode = 'w', encoding='utf-8') as file:
    for x in ids:
        try:
            tweet = api.get_status(x, tweet_mode='extended')
            json.dump(tweet._json, file)
            print(x)
        except:
            print('This tweet ID: {0}, raised an error.'.format(x))

892420643555336193
892177421306343426
891815181378084864
891689557279858688
891327558926688256
891087950875897856
890971913173991426
890729181411237888
890609185150312448
890240255349198849
890006608113172480
889880896479866881
889665388333682689
889638837579907072
889531135344209921
889278841981685760
888917238123831296
888804989199671297
888554962724278272
This tweet ID: 888202515573088257, raised an error.
888078434458587136
887705289381826560
887517139158093824
887473957103951883
887343217045368832
887101392804085760
886983233522544640
886736880519319552
886680336477933568
886366144734445568
886267009285017600
886258384151887873
886054160059072513
885984800019947520
885528943205470208
885518971528720385
885311592912609280
885167619883638784
884925521741709313
884876753390489601
884562892145688576
884441805382717440
884247878851493888
884162670584377345
883838122936631299
883482846933004288
883360690899218434
883117836046086144
882992080364220416
882762694511734784
88262727032160256

The JSON data is not parseable by the `json` decoder. [This stackoverflow answer](https://stackoverflow.com/questions/21708192/how-do-i-use-the-json-module-to-read-in-one-json-object-at-a-time/21709058#21709058) proved very useful parsing the following JSON data in `tweet_json.txt`. 

In [16]:
from functools import partial

def json_parse(fileobj, decoder=json.JSONDecoder(), buffersize=2048):
    buffer = ''
    for chunk in iter(partial(fileobj.read, buffersize), ''):
        buffer += chunk
        while buffer:
            try:
                result, index = decoder.raw_decode(buffer)
                yield result
                buffer = buffer[index:]
            except ValueError:
                break

Build a list of dictionaries using `json_parse()` to read the JSON objects individually, then create a new dataframe with our new information from the Twitter api.

In [33]:
df_list = []

with open('tweet_json.txt', mode='r', encoding='utf-8') as file:
    for data in json_parse(file):
        df_list.append({'rt_count': data['retweet_count'],
                        'fav_count': data['favorite_count'],
                        'tweet_id': data['id']})

In [34]:
cols = ['rt_count', 'fav_count', 'tweet_id']

from_api = pd.DataFrame(df_list,
                        columns = cols)

## Assessment

In [50]:
df.head(20)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rt_count,fav_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,8504.0,38536.0
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,6250.0,33037.0
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,4142.0,24876.0
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,8614.0,41947.0
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,9362.0,40075.0
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None,3104.0,20102.0
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None,2065.0,11769.0
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None,18841.0,65116.0
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None,4257.0,27631.0
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None,7387.0,31739.0


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
rt_count                      23

In [52]:
ip.head(20)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [102]:
ip.img_num.describe()

count    2075.000000
mean        1.203855
std         0.561875
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         4.000000
Name: img_num, dtype: float64

In [53]:
ip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [54]:
df[df.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rt_count,fav_count


In [55]:
ip[ip.duplicated()]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


In [59]:
ip['p1_conf'].describe()

count    2075.000000
mean        0.594548
std         0.271174
min         0.044333
25%         0.364412
50%         0.588230
75%         0.843855
max         1.000000
Name: p1_conf, dtype: float64

In [61]:
ip['p2_conf'].describe()

count    2.075000e+03
mean     1.345886e-01
std      1.006657e-01
min      1.011300e-08
25%      5.388625e-02
50%      1.181810e-01
75%      1.955655e-01
max      4.880140e-01
Name: p2_conf, dtype: float64

In [60]:
ip['p3_conf'].describe()

count    2.075000e+03
mean     6.032417e-02
std      5.090593e-02
min      1.740170e-10
25%      1.622240e-02
50%      4.944380e-02
75%      9.180755e-02
max      2.734190e-01
Name: p3_conf, dtype: float64

In [103]:
df.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator,rt_count,fav_count
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000,2343.000000,2343.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433,2982.078532,8036.713615
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237,4945.681841,12205.803767
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000,600.000000,1395.500000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000,1395.000000,3512.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000,3482.000000,9899.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000,76702.000000,142373.000000


## Quality issues

### Quality issues with the tweets dataframe

* Datatype issues:
 1. `tweet_id` should be a string not an int.
 2. `timestamp` should be a datetime object
* The `source` column has html tags still around it.
* The `in_reply_to...` columns should be dropped. They're almost entirely `NaN`s. The same can be said for the `retweeted_status...` columns.
* We don't want any data that is a retweet.

### Quality issues with the image predictions dataframe:
* `tweet_id` should be a string not an int
* `p1`, `p2`, and `p3` columns have '\_' instead of spaces. There's also inconsistency in their capitalization.
* We only want tweets with images in them. Some tweets don't have images.

### Api data issues
* `rt_count` and `fav_count` should be ints rather than floats.

## Tidiness issues

* The api dataframe and the enhanced tweets dataframe both have data on the same set of tweets, meaning each type of observational unit does *not* form a table.
* `doggo`, `floofer`, `pupper`, and `puppo` are not variables, but data. These shouldn't be in the columns.

## Cleaning

First copy the tweets dataframe before we do anything to it.

In [62]:
df_clean = df.copy()

Now tackle the quality issues in this dataframe.

#### *__`tweet_id` should be a string__*.

We'll fix this by using `.astype()`.

In [72]:
df_clean['tweet_id'] = df_clean['tweet_id'].astype(str)

Verify that the data type is a string:

In [73]:
df_clean['tweet_id'].dtype

dtype('O')

---------

#### *__`timestamp`__* should be a datetime object.

We'll fix this by using `pd.to_datetime`.

In [75]:
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])

Verify that the column is `datetime` type. This should show up as `<M8[ns]`.

In [77]:
df_clean['timestamp'].dtype

dtype('<M8[ns]')

---------------------

#### The `source` column still has html tags around it.

We grab everything inside the tag.

In [89]:
# We'll use regex to solve this problem
import re

In [85]:
# Let's us see all the contents of 'source'
pd.set_option('max_colwidth', 100)

Test and confirm our regex pattern works:

In [87]:
df_clean.source.head(100)

0     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
1     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
2     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
3     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
4     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
5     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
6     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
7     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
8     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
9     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
10    <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
11    <a href="http:/

In [94]:
pattern = re.compile(r'<a.*>([\w\s]+)</a>')

In [96]:
match = pattern.search('<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>')

In [97]:
match.groups()

('Twitter for iPhone',)

In [98]:
df_clean.source = df_clean.source.str.extract(pat = pattern)

C:\Users\Luke\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


Test to see if we've successfully gotten rid of the html tags:

In [99]:
df_clean.source.head()

0    Twitter for iPhone
1    Twitter for iPhone
2    Twitter for iPhone
3    Twitter for iPhone
4    Twitter for iPhone
Name: source, dtype: object

----------------------

`Retweeted_status...` columns should be dropped. So should `in_reply_to...` columns. 

# To do: Do we need to use these columns to filter out retweets? Drop all columns with non-null `retweeted_status..` column?

-------------


## Image predictions dataframe cleaning

First make a copy of image predictions dataframe.

In [104]:
ip_clean = ip.copy()

#### *__`tweet_id`__*  should be a string not an int.

In [105]:
ip_clean.tweet_id = ip_clean.tweet_id.astype(str)

Check datatype to ensure `tweet_id` is now a string:

In [107]:
ip_clean.tweet_id.dtype

dtype('O')

-------------

#### *__'

Merge the enhanced archive that was provided and the newly obtained data from the api to create our new dataframe with all the info we want.

In [48]:
df = pd.merge(ea, from_api, how='outer', on='tweet_id')